# 数字预处理
创建一个人工数据集，并存储在csv文件

In [19]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok = True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

从创建的csv文件中加载原始数据集

In [20]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


为了处理缺失的数据，典型的方法包括插值和删除，这里，我们考虑使用插值的方法
如果dataframe里有字符型的缺失值，直接用mean计算会报错，所以如果只计算数值型的均值，可以在mean时使用numeric_only=True

In [21]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean(numeric_only = True))
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


对于inputs中的类别值或离散值，我们将“NaN”视为一个类别
pd.get_dummies(df, columns=['X']) 会把X列拆成若干列，适用于分类变量。参数含义如下：
dummy_na=True：是否把 NaN 也当成一种分类值处理
dtype=int：把 one-hot 编码的列转换成整型（默认是 uint8）

In [23]:
inputs = pd.get_dummies(inputs, dummy_na = True, columns = ['Alley'], dtype = int)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


现在inputs和outputs中的所有条目都是数值类型，它们可以转换成张量格式

In [24]:
import torch

X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))